In [1]:
# ✅ Importing Libraries :
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, chi2, RFE

In [2]:
# ✅ 1.Loading Original Dataset : 
dataset=pd.read_csv("Preprocessed_Data_Cryptos - One Percent of Actual Dataset.csv",index_col=None)
print(dataset.shape)
dataset.head()

(5242, 18)


,Crypto,Date,Time - 24 Hour Format,Time - 12 Hour Format,volume,open,Close Minus Open,Adj Close Minus Open,Close Minus Adj Close,High Minus Open,High Minus Close,High Minus Adj Close,High Minus Low,close,adj_close,high,low,Trade Impact
0,SPC-USD,03-10-2020,00:00:00,12:00 AM,1954.0,0.005614,0.000301,0.000301,0.0,0.000305,0.000004,0.000004,0.000390,0.005915,0.005915,0.005919,0.005529,Positive Impact
1,CONX28135-USD,21-03-2025,00:00:00,12:00 AM,46719.0,17.543959,0.109655,0.109655,0.0,0.180546,0.070890,0.070890,0.242294,17.653614,17.653614,17.724504,17.482210,Positive Impact
2,SHX-USD,24-11-2020,00:00:00,12:00 AM,8190.0,0.000192,-0.000003,-0.000003,0.0,0.000019,0.000022,0.000022,0.000032,0.000189,0.000189,0.000211,0.000179,Negative Impact
3,PALLA-USD,16-12-2022,00:00:00,12:00 AM,84921.0,0.011293,0.000133,0.000133,0.0,0.000147,0.000014,0.000014,0.000162,0.011426,0.011426,0.011440,0.011278,Positive Impact
4,PORK29220-USD,16-05-2024,00:00:00,12:00 AM,1864946.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,No Impact


In [3]:
# ✅ 2. Encode 'Crypto'
from sklearn.preprocessing import LabelEncoder
import pickle

label_encoder = LabelEncoder()
dataset['Crypto_encoded'] = label_encoder.fit_transform(dataset['Crypto'])

In [4]:
# 1. Fit LabelEncoder
from sklearn.preprocessing import LabelEncoder
indep_X['Crypto'] = dataset['Crypto']
indep_X['Crypto_encoded'] = label_encoder.fit_transform(indep_X['Crypto'])

NameError: name 'indep_X' is not defined

In [ ]:
#Save the fitted encoder
import pickle
with open('prediction_app/crypto_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# ✅ 3. Convert 'Date' column to datetime and extract features
dataset['Date'] = pd.to_datetime(dataset['Date'], format='%d-%m-%Y')

# Extract features from date
dataset['Year'] = dataset['Date'].dt.year
dataset['Month'] = dataset['Date'].dt.month
dataset['Day'] = dataset['Date'].dt.day
dataset['DayOfWeek'] = dataset['Date'].dt.dayofweek
dataset['IsWeekend'] = dataset['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
#✅ 4.Duplicating the Original Dataset
dataset2 = dataset
dataset2.head()

In [ ]:
#✅ 4.Duplicating the Original Dataset
dataset2 = dataset2.drop(['Crypto','Date','Time - 24 Hour Format','Time - 12 Hour Format'],axis=1)
dataset2.head()

In [ ]:
#✅ 5.Classifying the Nominal Columns in Dataset : 
dataset2 = pd.get_dummies(dataset2, drop_first=True)
print(dataset2.shape)
dataset2.head()

In [ ]:
dataset2.columns

In [ ]:
#✅ 4.Assigning Variables (Independent/Dependent) : 

indep_X = dataset2.drop(['Trade Impact_Positive Impact'], axis=1)
print(indep_X.shape)

dep_Y = dataset2[['Trade Impact_Positive Impact']]
print(dep_Y.shape)

In [ ]:
#✅ 5.Creating Function(s) :

def train_test_split_and_StandardScaler(indep_X,dep_Y):

    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, Y_train, Y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, Y_train, Y_test

def RFE_Features_Classification(indep_X, dep_Y, n):

    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.svm import LinearSVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import SelectKBest, chi2, RFE
    
    RFE_List = []

    # Flatten y to a 1D array to avoid DataConversionWarning
    dep_Y = dep_Y.values.ravel()

    logistic_Regression = LogisticRegression(max_iter=1000, solver='lbfgs')
    #svc_Linear = SVC(kernel = 'linear', random_state = 0)
    svc_Linear = LinearSVC(dual=False, max_iter=10000, random_state=0)
    decisionTree = DecisionTreeClassifier(criterion = 'gini',max_features = 'sqrt',splitter = 'best', random_state = 0)
    randomForest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        
    RFE_Model_List = [logistic_Regression, svc_Linear, decisionTree, randomForest]

    for model in RFE_Model_List:
        print(f"\nRunning RFE for: {model}")
        start = time.time()
        
        logistic_RFE = RFE(estimator = model, n_features_to_select = n)
        logistic_RFE_Fit = logistic_RFE.fit(indep_X, dep_Y)
        logistic_RFE_Feature = logistic_RFE.transform(indep_X)
        RFE_List.append(logistic_RFE_Feature)
        print(f"Finished in {time.time() - start:.2f} seconds")
    return RFE_List

def Confusion_Matrix(classifier, X_test):
    y_pred = classifier.predict(X_test)

    from sklearn.metrics import confusion_matrix
    ConfusionMatrix = confusion_matrix(Y_test, y_pred)

    from sklearn.metrics import classification_report 
    ClassificationReport = classification_report(Y_test, y_pred)

    from sklearn.metrics import accuracy_score
    AccuracyScore=accuracy_score(Y_test, y_pred)         

    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore 

def Logistic_Regression(X_train,Y_train,X_test):       
    # Fitting K-NN to the Training set
    from sklearn.linear_model import LogisticRegression
    #classifier = LogisticRegression(random_state = 0) 
    classifier = LogisticRegression(solver='lbfgs', max_iter= 10000)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore   

def SVM_Linear(X_train,Y_train,X_test):

    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore

def SVM_Non_Linear(X_train,Y_train,X_test):

    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore

def Naive_Bayes(X_train,Y_train,X_test):       

    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore

def KNN(X_train,Y_train,X_test):

    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore

def DecisionTree(X_train,Y_train,X_test):

    # Fitting K-NN to the Training set
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore

def RandomForest(X_train,Y_train,X_test):

    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, Y_train)

    # Calling a Created Function - Confusion_Matrix(classifier,X_test) which returns - classifier, X_test, ConfusionMatrix, report, accuracy_score
    classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Confusion_Matrix(classifier, X_test)
    return classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore


def RFE_Classification(accuracy_LogisticRegression, accuracy_SVM_Linear, accuracy_SVM_NonLinear, 
                           accuracy_KNN, accuracy_NaiveBayes, accuracy_DecisionTree, accuracy_RandomForest): 

    dataframe=pd.DataFrame(index=['Logistic Regression','SVM Linear','Decision Tree','Random Forest'],
                           columns=['Logistic Regression','SVM Linear','SVM Non Linear','KNN','Naive Bayes',
                                    'Decision Tree','Random Forest'])
    
    #Function - enumerate() acts as a Counter which Iterates index starting from 0 (by default) and their item(s) from the iterable
    #Use enumerate() when We need both Position in the loop (number) and its value from the iterable (idex)
    
    for indexCount,indexValue in enumerate(dataframe.index):      
        dataframe.loc[indexValue, 'Logistic Regression'] = accuracy_LogisticRegression[indexCount]       
        dataframe.loc[indexValue, 'SVM Linear'] = accuracy_SVM_Linear[indexCount]
        dataframe.loc[indexValue, 'SVM Non Linear'] = accuracy_SVM_NonLinear[indexCount]
        dataframe.loc[indexValue, 'KNN'] = accuracy_KNN[indexCount]
        dataframe.loc[indexValue, 'Naive Bayes'] = accuracy_NaiveBayes[indexCount]
        dataframe.loc[indexValue, 'Decision Tree'] = accuracy_DecisionTree[indexCount]
        dataframe.loc[indexValue, 'Random Forest'] = accuracy_RandomForest[indexCount]
    return dataframe

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(indep_X)

In [ ]:
# Enter the Number of Features
No_Of_Features = 10

In [ ]:
Feature_List = []

# Loop through and print each regressor name with its value
for value in range(No_Of_Features, 0, -1):  
    print(f"\n<-------------------- Feature Count: {value} -------------------->")

    #✅ 6.Creating and Resetting accuracy lists for each feature count
    accuracy_LogisticRegression = []
    accuracy_SVM_Linear = []
    accuracy_SVM_NonLinear = []
    accuracy_KNN = []
    accuracy_NaiveBayes = []
    accuracy_DecisionTree = []
    accuracy_RandomForest = []

    #✅ 7.Calling a Created Function - RFE_Features_Classification(With Below Parameters): which returns - RFE_List
    RFE_List = RFE_Features_Classification(X_scaled, dep_Y, value)

    #✅ 8.Appending the Accuracy Score of All the Models in Created Empty List Through a For Loop 
    for X in RFE_List:
        X_train, X_test, Y_train, Y_test = train_test_split_and_StandardScaler(X, dep_Y)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Logistic_Regression(X_train, Y_train, X_test)
        accuracy_LogisticRegression.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = SVM_Linear(X_train, Y_train, X_test)  
        accuracy_SVM_Linear.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = SVM_Non_Linear(X_train, Y_train, X_test)  
        accuracy_SVM_NonLinear.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = KNN(X_train, Y_train, X_test)  
        accuracy_KNN.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = Naive_Bayes(X_train, Y_train, X_test)  
        accuracy_NaiveBayes.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = DecisionTree(X_train, Y_train, X_test)  
        accuracy_DecisionTree.append(AccuracyScore)
        
        classifier, X_test, Y_test, ConfusionMatrix, ClassificationReport, AccuracyScore = RandomForest(X_train, Y_train, X_test)  
        accuracy_RandomForest.append(AccuracyScore)

    #✅ 9.Calling a Created Function - RFE_Classification(With Below Parameters): which returns - dataframe    
    result = RFE_Classification(accuracy_LogisticRegression, accuracy_SVM_Linear, accuracy_SVM_NonLinear, 
                               accuracy_KNN, accuracy_NaiveBayes, accuracy_DecisionTree, accuracy_RandomForest)

    # Append a dictionary with feature count and result
    Feature_List.append({
        "Feature_Count": value,  # Number of features used
        "Result": result         # DataFrame of accuracy scores from multiple classifiers
    })

In [ ]:
# Combine all results into one DataFrame
combined_df = pd.concat([item["Result"].assign(Feature_Count = item["Feature_Count"]) for item in Feature_List], axis=0)

# Optional: reset index for cleanliness
combined_df = combined_df.reset_index()

# Convert all string columns to int (if safe)
combined_df = combined_df.apply(pd.to_numeric, errors='coerce')
combined_df = combined_df.drop(["index"], axis=1)

# Sort by Feature_Count in ascending order
sorted_df = combined_df.sort_values(by='Feature_Count', ascending=True)

# Format all float columns to 2 decimal places
formatted_df = sorted_df.round(2)
formatted_df

In [ ]:
# Calculate mode and drop Feature_Count column
sorted_mode = formatted_df.mode().drop("Feature_Count", axis=1)

# Remove all-NaN rows
clean_sorted_mode = sorted_mode.dropna(how='all')
clean_sorted_mode

In [ ]:
clean_sorted_mode_final = clean_sorted_mode.max()
clean_sorted_mode_final

In [ ]:
clean_sorted_mode_final_max = clean_sorted_mode_final.max()
clean_sorted_mode_final_max

In [ ]:
# Get all regressors with the maximum value
mode_max_final_items = clean_sorted_mode_final[clean_sorted_mode_final == clean_sorted_mode_final_max]

# Display names along with values
print(f"Final Classifier(s) with Highest Repeatation and Maximum Value(s):\n{mode_max_final_items}")

In [ ]:
# Feature Selection via RFE using Random Forest
rfe_selector = RFE(RandomForestClassifier(n_estimators=10, random_state=0), n_features_to_select=10)
rfe_selector = rfe_selector.fit(X_scaled, dep_Y)

In [ ]:
# Get selected features
selected_features = indep_X.columns[rfe_selector.get_support()]
print("Selected Features:\n", selected_features)

In [ ]:
# Apply same feature selection to full data
X_selected = indep_X[selected_features]

In [ ]:
# Split again on selected features
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split_and_StandardScaler(X_selected, dep_Y)

In [ ]:
# Scale again
sc_final = StandardScaler()
X_train_final_scaled = sc_final.fit_transform(X_train_final)
X_test_final_scaled = sc_final.transform(X_test_final)

In [ ]:
# Train final model
final_model = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
final_model.fit(X_train_final_scaled, y_train_final)

In [ ]:
import os

# Make the directory if it doesn't exist
os.makedirs("prediction_app", exist_ok=True)

In [ ]:
# Save the model
with open("prediction_app/model.pkl", "wb") as f:
    pickle.dump(final_model, f)

In [ ]:
# Save the scaler
with open("prediction_app/scaler.pkl", "wb") as f:
    pickle.dump(sc_final, f)

In [ ]:
# Save selected feature names
with open("prediction_app/selected_features.pkl", "wb") as f:
    pickle.dump(list(selected_features), f)